In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Interlub/orders_preprocessed.csv')
df['date'] = pd.to_datetime(df['date'])
# Agrupar por semana y por 'product_id', sumando solo 'quantity'
df_weekly = df.groupby([pd.Grouper(key='date', freq='W'), 'product_id'])['quantity'].sum().reset_index()
df_weekly.head()

## Models

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_squared_error
import numpy as np

from statsmodels.tsa.arima.model import ARIMA

from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

from statsforecast.models import AutoETS
from statsforecast import StatsForecast

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense

import logging
import warnings
warnings.filterwarnings("ignore")
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('fbprophet').setLevel(logging.WARNING)


def fitLinearRegression(product_id, product_series):
    product_series = product_series.copy()
    product_series['time_numeric'] = (product_series['ds'] - product_series['ds'].min()).dt.days
    X = product_series.index.values.reshape(-1, 1)
    y = product_series['y'].values  # Target variable
    model = LinearRegression()

    # Define Time Series Split
    tscv = TimeSeriesSplit(n_splits=5)
    rmse_scores = []

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = root_mean_squared_error(y_test, y_pred)
        rmse_scores.append(rmse)

    # Store results
    lr_results = [{
        'product_id': product_id,
        'LR_rmse': np.mean(rmse_scores),
    }]
    return pd.DataFrame(lr_results)


def fitArima(product_id, product_series, p,d,q):
    product_series = product_series.copy()
    product_series = product_series.drop(columns=['ds'])
    tscv = TimeSeriesSplit(n_splits=5)
    rmse_scores = []

    for train_index, test_index in tscv.split(product_series):
        train, test = product_series.iloc[train_index], product_series.iloc[test_index]

        # Fit ARIMA model
        model = ARIMA(train, order=(p, d, q))  # Adjust (p, d, q) as needed
        model_fit = model.fit()

        # Predict and evaluate
        predictions = model_fit.forecast(steps=len(test))
        rmse = root_mean_squared_error(test, predictions)
        rmse_scores.append(rmse)

    arima_results = [{
        'product_id': product_id,
        'ARIMA_rmse': np.mean(rmse_scores),
    }]
    return pd.DataFrame(arima_results)


def fitProphet(product_id, product_series):
    product_series = product_series.copy()
    modelo = Prophet()
    modelo.fit(product_series)
    futuro = modelo.make_future_dataframe(periods=5)
    prediccion = modelo.predict(futuro)

    #Cross-validation
    df_cv = cross_validation(modelo, initial='540 days', period='90 days', horizon='180 days')
    rmse_scores = performance_metrics(df_cv)['rmse']

    prophet_results = [{
        'product_id': product_id,
        'PROPHET_rmse': np.mean(rmse_scores),
    }]
    return pd.DataFrame(prophet_results)

def fitProphetLog(product_id, product_series):
    product_series = product_series.copy()

    product_series['cap'] = product_series['y'].max()
    product_series['floor'] = 0

    modelo = Prophet(growth='logistic')
    modelo.fit(product_series)
    futuro = modelo.make_future_dataframe(periods=5)
    futuro['cap'] = product_series['y'].max()
    futuro['floor'] = 0

    prediccion = modelo.predict(futuro)

    #Cross-validation
    df_cv = cross_validation(modelo, initial='540 days', period='90 days', horizon='180 days')
    rmse_scores = performance_metrics(df_cv)['rmse']

    prophetlog_results = [{
        'product_id': product_id,
        'PROPHETLOG_rmse': np.mean(rmse_scores),
    }]

    return pd.DataFrame(prophetlog_results)

def fitAutoEts(product_id, product_series):
    sf_df = product_series.copy()
    sf_df['unique_id'] = product_id  # StatsForecast requires this column
    models = [AutoETS(season_length=52, model='ZZZ', alias='AutoETS')]
    sf = StatsForecast(models=models, freq='W', n_jobs=-1)

    cv_df = sf.cross_validation(
        df=sf_df,
        h=12,           # Forecast horizon (12 weeks ~ 3 months)
        n_windows=5,     # 5 folds of cross-validation
        step_size=24,    # Move 24 weeks (~6 months) between windows
        test_size=156-24 # Ensure we have enough training data (132 weeks for first fold)
    )

    # Calculate RMSE for our specific model
    mean_fold_rmse = cv_df.groupby('cutoff').apply(
        lambda x: np.sqrt(mean_squared_error(x['y'], x['AutoETS']))).mean()

    return pd.DataFrame([{
        'product_id': product_id,
        'AUTOETS_rmse': mean_fold_rmse,
    }])



def fitLSTM(product_id, product_series, look_back=10, epochs=20, batch_size=32):
    # Prepare data
    data = product_series[['y']].values
    dates = product_series['ds'].values

    if len(data) <= look_back * 2:  # Need enough data for sequences
        return pd.DataFrame([{
            'product_id': product_id,
            'LSTM_rmse': np.nan,
        }])

    # Normalize data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Create sequences
    def create_sequences(dataset, look_back):
        X, y = [], []
        for i in range(len(dataset) - look_back):
            X.append(dataset[i:i+look_back, 0])
            y.append(dataset[i+look_back, 0])
        return np.array(X), np.array(y)

    X, y = create_sequences(scaled_data, look_back)
    X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape for LSTM

    # Time Series Cross-Validation
    tscv = TimeSeriesSplit(n_splits=5)
    rmse_scores = []

    for train_index, test_index in tscv.split(X):
        try:
            # Split data
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Build LSTM model
            model = Sequential()
            model.add(LSTM(50, input_shape=(look_back, 1)))
            model.add(Dense(1))
            model.compile(loss='mean_squared_error', optimizer='adam')

            # Train
            model.fit(X_train, y_train,
                     epochs=epochs,
                     batch_size=batch_size,
                     verbose=0)

            # Predict and inverse transform
            predictions = model.predict(X_test, verbose=0)
            predictions = scaler.inverse_transform(predictions)
            y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

            # Calculate RMSE
            rmse = np.sqrt(mean_squared_error(y_test, predictions))
            rmse_scores.append(rmse)

        except Exception as e:
            print(f"Error in fold for product {product_id}: {str(e)}")
            rmse_scores.append(np.nan)
            continue

    # Return results in same format as other models
    return pd.DataFrame([{
        'product_id': product_id,
        'LSTM_rmse': np.nanmean(rmse_scores),  # Handles any NaN folds
    }])

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_squared_error
import numpy as np

products = df_weekly['product_id'].unique()
all_results = []
final_metrics = pd.DataFrame()

for product_id in products:
    product_df = df_weekly[df_weekly['product_id'] == product_id].copy()
    product_df = product_df.sort_values('date')
    product_df = product_df.reset_index(drop=True)
    product_df = product_df.drop(columns=['product_id'])
    product_df = product_df.rename(columns={'date': 'ds', 'quantity': 'y'})

    lr_results = fitLinearRegression(product_id, product_df)
    arima_results = fitArima(product_id, product_df, p=1, d=0, q=1)
    prophet_results = fitProphet(product_id, product_df)
    prophet_log_results = fitProphetLog(product_id, product_df)
    autoets_results = fitAutoEts(product_id, product_df)
    lstm_results = fitLSTM(product_id, product_df)

    product_results = prophet_results.merge(lr_results, on='product_id', how='outer').merge(
                      arima_results, on='product_id', how='outer').merge(
                      prophet_log_results, on='product_id', how='outer').merge(
                      autoets_results, on='product_id', how='outer').merge(
                      lstm_results, on='product_id', how='outer')

    all_results.append(product_results)

final_metrics = pd.concat(all_results, ignore_index=True)

In [ ]:
pd.set_option("display.max_rows", None)
final_metrics

In [ ]:
mean_values = final_metrics.select_dtypes(include=['number']).mean()
print("Mean values for each metric:")
print(mean_values)

Mean values for each metric:
PROPHET_rmse       268.822707
LR_rmse            258.887531
ARIMA_rmse         242.225624
PROPHETLOG_rmse    305.205552
AUTOETS_rmse       232.263085
LSTM_rmse          241.924647
dtype: float64
